### 데이터 로깅 실습하기

In [1]:
import os
import wandb

wandb.login()
wandb.init(project="trace-example")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 40kimwoosung (40kimwoosung-seoul-national-university). Use `wandb login --relogin` to force relogin


In [ ]:
import datetime
from openai import OpenAI
from wandb.sdk.data_types.trace_tree import Trace

os.environ["OPENAI_API_KEY"] = "Your OpenAI API key"

system_message = "You are a helpful assistant."
query = "대한민국의 수도는 어디야?"

temperature = 0.2
model_name = 'gpt-3.5-turbo'

response = client.chat.completions.create(
    model=model_name,
    messages=[
        {
            'role': 'system',
            'content': system_message,
        },
        {
            'role': 'user',
            'content': query
        }
    ],
    temperature=temperature
)

root_span = Trace(
    name='root_span',
    kind='llm',
    status_code='success',
    status_message=None,
    metadata={
        'temperature': temperature,
        'token_usage': dict(response.usage),
        'model_name': model_name
    },
    inputs={
        'system_prompt': system_message,
        'query': query
    },
    outputs={
        'response': response.choices[0].message.content
    },
)

root_span.log(name='openai_trace')

In [ ]:
from datasets import load_dataset
import llama_index
from llama_index.core import Document, VectorStoreIndex, Settings
from llama_index.llms.openai import OpenAI
from llama_index.core import set_global_handler

# 로깅을 위한 설정 추가
llm = OpenAI(model='gpt-3.5-turbo', temperature=0)
set_global_handler('wandb', run_args={'project': 'llamaindex'})
wandb_callback = llama_index.core.global_handler
Settings.llm = llm

dataset = load_dataset('klue', 'mrc', split='train')
text_list = dataset[:100]['context']
documents = [Document(text=text) for text in text_list]

index = VectorStoreIndex.from_documents(documents)

print(dataset[0]['question']) 

query_engine = index.as_query_engine(llm=llm, similarity_top_k=1, verbose=True)
response = query_engine.query(
    dataset[0]['question']
)

wandb: Logged trace tree to W&B.


북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?


wandb: Logged trace tree to W&B.
